# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive import MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'udacity-project'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [3]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--C": choice(1,2,3,4), "--max_iter": choice(40,80,120,130,200), "--n_jobs":choice(1,2)})

compute_target = 'udacity'

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='./', compute_target=compute_target, entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, policy=early_termination_policy, primary_metric_name="Accuracy", 
hyperparameter_sampling=param_sampling, max_total_runs=10, primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
#TODO: Submit your experiment
hyper_drive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [5]:
RunDetails(hyper_drive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [6]:
best_run = hyper_drive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength:',best_run_metrics['Regularization:'])
print('\n Max Iterations:',best_run_metrics['Maximum Iterations:'])

Best Run Id:  HD_6939e92e-0cd0-4def-a754-d09768a0ed91_1

 Accuracy: 0.7240215924426451

 Regularization Strength: 2.0

 Max Iterations: 120


In [7]:
#TODO: Save the best model
model = best_run.register_model(model_name='hyper_drive_best_model', model_path='./', 
                                properties={'Accuracy': best_run_metrics['Accuracy'],
                                            'Regularization Strength': best_run_metrics['Regularization:'],
                                            'Max Iterations': best_run_metrics['Maximum Iterations:']})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service